# setup api and dataset

In [1]:
!pip install goodfire


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/76.4 kB ? eta -:--:--
     ---------------------                    41.0/76.4 kB ? eta -:--:--
     ---------------------------------------- 76.4/76.4 kB 1.4 MB/s eta 0:00:00
                                              0.0/139.8 kB ? eta -:--:--
     ------------------------------------   133.1/139.8 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 139.8/139.8 kB 2.8 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
                                              0.0/455.3 kB ? eta -:--:--
     -------------                          163.8/455.3 kB 5.0 MB/s eta 0:00:01
     --------------------------             317.4/455.3 kB 3.9 MB/s eta 0:00:01
     -------------------------------------- 455.3/455.3 kB 4.1 MB/s eta 0:00:00
                                              0.0/90.4 kB ? eta -:--:--
     ---------------------------------------- 90.4/90.4 kB ? eta 0:00:00
                    

In [2]:
try:
  from google.colab import userdata

  # Add your Goodfire API Key to your Colab secrets
  GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')
except:
  keyfile = open("../goodfire_apikey.txt", "r")
  GOODFIRE_API_KEY = keyfile.readline()

import goodfire
client = goodfire.Client(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
variant_small = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")
variant_large = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")

In [3]:
# first try to install datasets
!pip install datasets

                                              0.0/480.6 kB ? eta -:--:--
     ------------                           153.6/480.6 kB 3.1 MB/s eta 0:00:01
     -------------------------              317.4/480.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------  471.0/480.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 480.6/480.6 kB 3.0 MB/s eta 0:00:00
                                              0.0/25.1 MB ? eta -:--:--
                                              0.1/25.1 MB 3.5 MB/s eta 0:00:08
                                              0.3/25.1 MB 4.1 MB/s eta 0:00:07
                                              0.6/25.1 MB 4.3 MB/s eta 0:00:06
     -                                        0.8/25.1 MB 4.6 MB/s eta 0:00:06
     -                                        0.9/25.1 MB 4.6 MB/s eta 0:00:06
     -                                        1.1/25.1 MB 4.2 MB/s eta 0:00:06
     --                                       1.3/25.1 MB 4.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.26.2 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# try tqdm (it doesnt seem to work for some reason)
!pip install tqdm
from tqdm.notebook import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.01)


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# next try to download the triva dataset
from datasets import load_dataset

ds = load_dataset("mandarjoshi/trivia_qa", "unfiltered.nocontext")

# Run the Baseline 

In [ ]:
# try running through all of the training set to see what performance we get at base
variant_small.reset()
prompt = "You are an AI assitant answering trivia questions. Give a single concise answer."
num_correct = 0
timeout = 2
for index, sample in enumerate(tqdm(ds["train"])):
    sample_question =  sample["question"]
    sample_answers = sample['answer']['normalized_aliases']

    # now get the model response
    response = client.chat.completions.create(
        [
            {"role": "system", "content": prompt},
            {"role": "user", "content": sample_question}
        ],
        model=variant_small,
        stream=False,
        max_completion_tokens=50,
    )

    given_answer = response.choices[0].message["content"].lower()

    #given_answer = ""
    #for token in response:
    #    given_answer += token.choices[0].delta.content
    #given_answer = given_answer.lower()

    for answer in sample_answers:
        if answer in given_answer:
            num_correct += 1
            break

    if index > 300:
        break

    # make sure not to spam the api
    time.sleep(timeout)


In [ ]:
print(num_correct)
print(index)
print(num_correct/index)

In [ ]:
# try running through all of the training set to see what performance we get at base
variant_large.reset()
prompt = "You are an AI assitant answering trivia questions. Give a single concise answer."
num_correct = 0
timeout = 2
for index, sample in enumerate(tqdm(ds["train"])):
    sample_question =  sample["question"]
    sample_answers = sample['answer']['normalized_aliases']

    # now get the model response
    response = client.chat.completions.create(
        [
            {"role": "system", "content": prompt},
            {"role": "user", "content": sample_question}
        ],
        model=variant_large,
        stream=False,
        max_completion_tokens=50,
    )

    given_answer = response.choices[0].message["content"].lower()

    #given_answer = ""
    #for token in response:
    #    given_answer += token.choices[0].delta.content
    #given_answer = given_answer.lower()

    for answer in sample_answers:
        if answer in given_answer:
            num_correct += 1
            break

    if index > 300:
        break

    # make sure not to spam the api
    time.sleep(timeout)


In [ ]:
print(num_correct)
print(index)
print(num_correct/index)